In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
ntire2017_train_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/train'
ntire2017_val_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/val'
ntire2017_track1_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/train/DIV2K_train_LR_bicubic_X2'
ntire2017_track2_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/train/DIV2K_train_LR_unknown_X2'
ntire2017_hr_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/train/DIV2K_train_HR'
ntire2017_val_track1_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/val/DIV2K_valid_LR_bicubic_X2'
ntire2017_val_track2_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/val/DIV2K_valid_LR_unknown_X2'
ntire2017_val_hr_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/val/DIV2K_valid_HR'
ntire2018_train_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train'
ntire2018_val_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val'
ntire2018_track1_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_LR_x8'
ntire2018_track2_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_LR_mild'
ntire2018_track3_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_LR_difficult'
ntire2018_track4_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_LR_wild'
ntire2018_hr_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_HR'
ntire2018_val_track1_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_LR_x8'
ntire2018_val_track2_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_LR_mild'
ntire2018_val_track3_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_LR_difficult'
ntire2018_val_track4_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_LR_wild'
ntire2018_val_hr_dir = '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_HR'

In [4]:
# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator_2017 = train_datagen.flow_from_directory(
    ntire2017_train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)
train_generator_2018 = train_datagen.flow_from_directory(
    ntire2018_train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator_2017 = test_datagen.flow_from_directory(
    ntire2017_val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)
validation_generator_2018 = test_datagen.flow_from_directory(
    ntire2018_val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)


Found 2214 images belonging to 3 classes.
Found 5860 images belonging to 5 classes.
Found 254 images belonging to 3 classes.
Found 500 images belonging to 5 classes.


In [5]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [ ]:
epochs = 10

history = model.fit(
    train_generator_2017,
    steps_per_epoch=train_generator_2017.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator_2017,
    validation_steps=validation_generator_2017.samples // batch_size
)
history = model.fit(
    train_generator_2018,
    steps_per_epoch=train_generator_2018.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator_2018,
    validation_steps=validation_generator_2018.samples // batch_size
)


Epoch 1/10
69/69 [==============================] - 1243s 18s/step - loss: -14773.6426 - accuracy: 0.3602 - val_loss: 164977.2812 - val_accuracy: 0.3438
Epoch 2/10
69/69 [==============================] - 459s 7s/step - loss: -1658602.0000 - accuracy: 0.3621 - val_loss: 18920878.0000 - val_accuracy: 0.3170
Epoch 3/10
69/69 [==============================] - 471s 7s/step - loss: -37123272.0000 - accuracy: 0.3616 - val_loss: 132584408.0000 - val_accuracy: 0.3170
Epoch 4/10
69/69 [==============================] - 467s 7s/step - loss: -270712256.0000 - accuracy: 0.3607 - val_loss: 976861184.0000 - val_accuracy: 0.3214
Epoch 5/10
69/69 [==============================] - 458s 7s/step - loss: -1104878976.0000 - accuracy: 0.3630 - val_loss: 1602791040.0000 - val_accuracy: 0.3482
Epoch 6/10
69/69 [==============================] - 467s 7s/step - loss: -3699433984.0000 - accuracy: 0.3616 - val_loss: 6618738176.0000 - val_accuracy: 0.3304
Epoch 7/10
69/69 [==============================] - 450s 

In [ ]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(validation_generator_2017)
print(f'Validation accuracy (NTIRE 2017): {accuracy * 100:.2f}%')

loss, accuracy = model.evaluate(validation_generator_2018)
print(f'Validation accuracy (NTIRE 2018): {accuracy * 100:.2f}%')


In [ ]:
import tensorflow as tf

model.save('/content/my_model')

In [ ]:
pip install opencv-python-headless tensorflow numpy


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Define directories
directories = {
    'ntire2017_train': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/train',
    'ntire2017_val': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/val',
    'ntire2017_track1': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/train/DIV2K_train_LR_bicubic_X2',
    'ntire2017_track2': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/train/DIV2K_train_LR_unknown_X2',
    'ntire2017_hr': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/train/DIV2K_train_HR',
    'ntire2017_val_track1': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/val/DIV2K_valid_LR_bicubic_X2',
    'ntire2017_val_track2': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/val/DIV2K_valid_LR_unknown_X2',
    'ntire2017_val_hr': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2017/val/DIV2K_valid_HR',
    'ntire2018_train': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train',
    'ntire2018_val': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val',
    'ntire2018_track1': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_LR_x8',
    'ntire2018_track2': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_LR_mild',
    'ntire2018_track3': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_LR_difficult',
    'ntire2018_track4': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_LR_wild',
    'ntire2018_hr': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/train/DIV2K_train_HR',
    'ntire2018_val_track1': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_LR_x8',
    'ntire2018_val_track2': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_LR_mild',
    'ntire2018_val_track3': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_LR_difficult',
    'ntire2018_val_track4': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_LR_wild',
    'ntire2018_val_hr': '/content/drive/MyDrive/Colab Notebooks/INTEL/dataset/ntire2018/val/DIV2K_valid_HR'
}

# Function to load and preprocess images
def load_and_preprocess_images(directory, target_size=(1920, 1080)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, target_size)
                image = image / 255.0  # Normalize to [0, 1]
                images.append(image)
    return np.array(images)

# Load and preprocess images from all directories
all_images = {}
for key, directory in directories.items():
    all_images[key] = load_and_preprocess_images(directory)

# Example: Accessing images from ntire2017_train
ntire2017_train_images = all_images['ntire2017_train']
print(f"Loaded {len(ntire2017_train_images)} images from ntire2017_train")


In [ ]:
# Save preprocessed data
for key, images in all_images.items():
    np.save(f'/content/drive/MyDrive/Colab Notebooks/INTEL/preprocessed_{key}.npy', images)


In [ ]:
from google.colab import files
from IPython.display import HTML

# Upload the HTML file to Colab
uploaded = files.upload()

# Display the HTML UI
HTML(uploaded['pixelation-detection-ui.html'])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')